In [1]:
import os
from dotenv import load_dotenv
import psycopg2

# Charger les variables d'environnement
load_dotenv(".env")

# Obtenir les informations d'identification de la base de données depuis le fichier .env
USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

try:
    conn = psycopg2.connect(
        user=USER_PSQL,
        password=PASSWORD_PSQL,
        host="localhost",
        port="5432",
        dbname="app_auth"
    )
    print("Connexion à la base de données établie avec succès.")
except Exception as e:
    print(f"Erreur de connexion à la base de données: {e}")


Connexion à la base de données établie avec succès.


## Question 1: Création de la Table `user_for_comparison`

Nous allons créer une table `user_for_comparison` qui a la même structure que la table `user`. Cette table sera utilisée pour comparer les performances des requêtes avant et après l'ajout d'un index.

In [4]:
create_table_script = """
CREATE TABLE IF NOT EXISTS user_for_comparison (
    user_id SERIAL PRIMARY KEY,
    firstname VARCHAR(255),
    lastname VARCHAR(255),
    email VARCHAR(255) UNIQUE,
    username VARCHAR(255) UNIQUE,
    password VARCHAR(255),
    created_at TIMESTAMP WITHOUT TIME ZONE
);
"""
cursor = conn.cursor()
cursor.execute(create_table_script)
conn.commit()

### Insertion des Données

Nous allons maintenant insérer 100 000 lignes de données générées de manière crédible dans la table `user_for_comparison`.

In [4]:
from faker.generator import random
from faker import Faker
import psycopg2

faker = Faker()
cursor = conn.cursor()

for _ in range(100):  # 100 lots de 1000 utilisateurs
    for _ in range(1000):
        try:
            cursor.execute(
                "INSERT INTO user_for_comparison (firstname, lastname, email, username, password, created_at) VALUES (%s, %s, %s, %s, %s, %s)",
                (
                    faker.first_name(),
                    faker.last_name(),
                    faker.email(),
                    faker.user_name() + str(random.randint(1, 1000000)),  # Ajouter un nombre aléatoire pour l'unicité
                    faker.password(),
                    faker.date_between(start_date="-2y", end_date="today")
                )
            )
        except psycopg2.errors.UniqueViolation:
            conn.rollback()  # Annule l'insertion en cas de doublon
        else:
            conn.commit()  # Valide l'insertion si aucune erreur



### Question 2: Mesure du Temps d'Exécution des Requêtes

Nous allons mesurer le temps d'exécution des requêtes `SELECT` avec un filtre `WHERE` sur différents `username` dans la table `user_for_comparison`.

In [3]:
import time

usernames_to_test = ["example_username1", "example_username2", "example_username3"]
cursor = conn.cursor()
for username in usernames_to_test:
    start_time = time.time()
    cursor.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    cursor.fetchall()
    end_time = time.time()

    print(f"Temps d'exécution pour {username}: {end_time - start_time} secondes")

Temps d'exécution pour example_username1: 0.003629922866821289 secondes
Temps d'exécution pour example_username2: 0.0 secondes
Temps d'exécution pour example_username3: 0.0 secondes


### Question 3: Création d'un Index sur la Colonne `username`

Nous allons créer un index sur la colonne `username` de la table `user_for_comparison` et répéter les mesures de temps d'exécution des requêtes pour comparer les performances.

In [4]:
# Création de l'index
create_index_script = "CREATE INDEX idx_username ON user_for_comparison (username);"
cursor.execute(create_index_script)
conn.commit()

# Mesurer à nouveau le temps d'exécution pour les mêmes usernames
for username in usernames_to_test:
    start_time = time.time()
    cursor.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    cursor.fetchall()
    end_time = time.time()

    print(f"Temps d'exécution après indexation pour {username}: {end_time - start_time} secondes")


Temps d'exécution après indexation pour example_username1: 0.0010018348693847656 secondes
Temps d'exécution après indexation pour example_username2: 0.0009996891021728516 secondes
Temps d'exécution après indexation pour example_username3: 0.0 secondes
